In [5]:
pip install python_utils

  Obtaining dependency information for python_utils from https://files.pythonhosted.org/packages/26/45/98431ba6d17b99468bd3f4c53fdeefff402167f006a06773905296f6d489/python_utils-3.8.2-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets

# custom logger from (https://raw.githubusercontent.com/diegoalejogm/gans/master/utils.py) 
from utils import Logger

import numpy as np
import imageio as img
import math
import time
import os

# Helper Functions

In [61]:
def images_to_vectors(images, length):
    # reshapes the input iamge into a vector of given length
    # just checking that information isnt lost in the covnersion
    if np.prod(images.size()[1:]) > length:
        length = np.prod(images.size()[1:])
    return images.view(images.size(0), length)

def vectors_to_images(vectors, channel, width, height):
    # just error checking
    assert np.prod(images.size()[1:]) == channel*width*height
    return vectors.view(vectors.size(0), channel, width, height)

def noise(size, batchSize):
    n = Variable(torch.randn(size, batchSize))
    if torch.cuda.is_available(): return n.cuda() 
    return n

def ones_target(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1))
    return data

def zeros_target(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1))
    return data

In [19]:
# unlike tensorflow where we have to define the graphs and their operations, in pytorch we deine the network as a class
# i.e we just define the operations and the forward prop, the backprop is sorta in built

# we get this done by defining our class that inherits fromthe torchnn.module and hence the other functions
# such as backprop can be used

In [62]:
# hyper parameters
bSize = 100
learning_rate = 0.0002

features = 784
# Number of steps to apply to the discriminator
d_steps = 1  # In Goodfellow et. al 2014 this variable is assigned to 1
# Number of epochs
num_epochs = 200


# for testing 
num_test_samples = 16
test_noise = noise(num_test_samples, bSize)

# Data Processing

In [22]:
# setting variables for the directory to the data
todata = "/home/abdullah/Desktop/Abdullah/LUMS/Senior/Sproj/ImgPreProcessing/Channel/"
moredata = "/home/abdullah/Desktop/Abdullah/LUMS/Senior/Sproj/ImgPreProcessing/AmplifiedDataset"

norm = 255.0/2

In [52]:
# saving current directory
cur = os.getcwd()

# changing directoy to data set
os.chdir(todata)
# getting all the folder names
nmes = os.listdir(".")
nmes.sort()

# reading all the data into labels and data numpy arrays
labels = []
data = []
for letr in nmes:
    for file in os.listdir(todata+letr):
        labels.append(letr)
        f = img.imread(todata + letr + "/"+ file)
#         f = f.reshape(-1, 1)
#         data.append((torch.from_numpy(f), torch.from_numpy(np.array(1))))
        data.append(f)
#         print(f.shape)
#         plt.imshow(f)
#         plt.show()
#         break
#     break
        # change back directory
os.chdir(cur)

print("\nData labales matrix")
labels = np.array(labels)
print(labels, labels.dtype, labels.shape)

# diving by 255 to get clamp values between zero and one
print("\nData matrix")
# GAN hack to keep data betwen -1-1
data = (np.array(data, np.float32)/norm)-1
print(data.dtype, data.shape)



Data labales matrix
['A' 'A' 'A' ... 'Z' 'Z' 'Z'] <U1 (3844,)

Data matrix


ValueError: only one element tensors can be converted to Python scalars

In [5]:
DATA_FOLDER = "./mnist"

In [6]:
# normailze data (https://github.com/soumith/ganhacks)
def mnist_data():
    compose = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((.5, .5, .5), (.5, .5, .5))
        ])
    out_dir = '{}/dataset'.format(DATA_FOLDER)
    return datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)

In [7]:
# Load data
data = mnist_data()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████| 9912422/9912422 [00:00<00:00, 10300402.73it/s]


Extracting ./mnist/dataset/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████| 28881/28881 [00:00<00:00, 385107.91it/s]


Extracting ./mnist/dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|████████████████████████████| 1648877/1648877 [00:00<00:00, 3638487.45it/s]


Extracting ./mnist/dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████| 4542/4542 [00:00<00:00, 7623260.81it/s]

Extracting ./mnist/dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/dataset/MNIST/raw



In [9]:
bSize = 100

# Create loader with data, so that we can iterate over it
data_loader = torch.utils.data.DataLoader(data, batch_size=bSize, shuffle=True)
# Num batches
num_batches = len(data_loader)

# Discriminator

In [64]:
# leaky relu used to avoid sparse gradients (https://github.com/soumith/ganhacks)
class LeDiscriminator(torch.nn.Module):
    ## three hidden layer network, 
    ## nfeat will be flattend images eg 28x28 = 784
    ## output is 1 since its just learns to distinguish between real and fake
    def __init__(self, n_feat):
        super().__init__()
        # input vector
        n_features = n_feat
        # binary output
        n_out = 1

        self.hidden0 = nn.Sequential( 
            nn.Linear(n_features, 1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        self.out = nn.Sequential(
            torch.nn.Linear(256, n_out),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x

# Generator Network

In [65]:
class LeGenereator(torch.nn.Module):

    def __init__(self, batchSize, vecOut):
        super().__init__()
        # number of images to produce
        n_features = batchSize
        # length of vector that will be input to the Discriminator
        n_out = vecOut
        
        self.hidden0 = nn.Sequential(
            nn.Linear(n_features, 256),
            nn.LeakyReLU(0.2)
        )
        self.hidden1 = nn.Sequential(            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2)
        )
        self.hidden2 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2)
        )
        
        self.out = nn.Sequential(
            nn.Linear(1024, n_out),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x
    

# Defining Network Essentials

In [66]:
discriminator = LeDiscriminator(features)
generator = LeGenereator(bSize, features)
if torch.cuda.is_available():
    discriminator.cuda()
    generator.cuda()

In [67]:
# Optimizers
d_optimizer = optim.Adam(discriminator.parameters(), lr=learning_rate)
g_optimizer = optim.Adam(generator.parameters(), lr=learning_rate)

# Loss function
loss = nn.BCELoss()

# Train Functions

In [74]:
def real_data_target(size):
    '''
    Tensor containing ones, with shape = size
    '''
    data = Variable(torch.ones(size, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

def fake_data_target(size):
    '''
    Tensor containing zeros, with shape = size
    '''
    data = Variable(torch.zeros(size, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

In [68]:
def train_discriminator(optimizer, real_data, fake_data):
    # Reset gradients
    optimizer.zero_grad()
    
    # 1.1 Train on Real Data
    prediction_real = discriminator(real_data)
    # Calculate error and backpropagate
    error_real = loss(prediction_real, real_data_target(real_data.size(0)))
    error_real.backward()

    # 1.2 Train on Fake Data
    prediction_fake = discriminator(fake_data)
    # Calculate error and backpropagate
    error_fake = loss(prediction_fake, fake_data_target(real_data.size(0)))
    error_fake.backward()
    
    # 1.3 Update weights with gradients
    optimizer.step()
    
    # Return error
    return error_real + error_fake, prediction_real, prediction_fake


In [69]:
def train_generator(optimizer, fake_data):
    # Reset gradients
    optimizer.zero_grad()
    
    # Sample noise and generate fake data
    prediction = discriminator(fake_data)
    
    # Calculate error and backpropagate
    error = loss(prediction, real_data_target(prediction.size(0)))
    error.backward()
    
    # Update weights with gradients
    optimizer.step()
    
    # Return error
    return error

# Actual Training

In [76]:
for epoch in range(num_epochs):
    for n_batch, (real_batch,_) in enumerate(data_loader):
        N = real_data.size(0)
        # 1. Train Discriminator
        real_data = Variable(images_to_vectors(real_batch, features))
        if torch.cuda.is_available(): real_data = real_data.cuda()
        # Generate fake data and detach 
        # (so gradients are not calculated for generator)
        # can use.clone() as well but it will be a copy
        # .detach() uses the same memory
        fake_data = generator(noise(N, bSize)).detach()
        # Train D
        d_error, d_pred_real, d_pred_fake = \
              train_discriminator(d_optimizer, real_data, fake_data)

        # 2. Train Generator
        # Generate fake data
        fake_data = generator(noise(N, bSize))
        # Train G
        g_error = train_generator(g_optimizer, fake_data)
#         # Log batch error
#         logger.log(d_error, g_error, epoch, n_batch, num_batches)
        # Display Progress every few batches
    
        if (n_batch) % 100 == 0: 
            test_images = vectors_to_images(generator(test_noise))
            test_images = test_images.data

#             logger.log_images(
#                 test_images, num_test_samples, 
#                 epoch, n_batch, num_batches
#             );
            # Display status Logs
#             logger.display_status(
#                 epoch, num_epochs, n_batch, num_batches,
#                 d_error, g_error, d_pred_real, d_pred_fake
#             )

TypeError: vectors_to_images() missing 3 required positional arguments: 'channel', 'width', and 'height'